In [11]:
import pandas as pd
import re
import fuzzy_pandas as fpd
import fuzzymatcher

All_MPs = pd.read_excel("all_MPs_bd_April7-2.xls")
#imported it as an xls because csv's were doing funny things to the names 
All_MPs.head(2)


,Name,Type.of.Parliamentarian,Riding.Senatorial.Division,Province.Territory,Gender,Political.Affiliation,Full,Start_date,End_date,Birth.Date.Combined
0,"Lind, James (Jim) Gordon",MP (1968/06/25 - 1972/10/29)\nMP (1965/11/08 -...,Middlesex (1968/06/25 - 1972/10/29)\nMiddlesex...,Ontario\nOntario,M,Liberal Party of Canada (1965/11/08 - 1972/10/29),James Lind,1965-11-08 00:00:00,1972-10-29 00:00:00,1913-03-08 00:00:00
1,"Hébert, Jacques",Senator (1983/04/20 - 1998/06/21),Wellington (1983/04/20 - 1998/06/21),Quebec,M,Liberal Party of Canada (1983/04/20 - 1998/06/21),Jacques Hébert,1983-04-20 00:00:00,1998-06-21 00:00:00,1923-06-21 00:00:00


In [12]:
MPs_parliament = pd.read_csv ('MPs_parliament.csv')
MPs_parliament.head(2)


,Name,number
0,"Beaubien, Arthur-Lucien",14
1,"Beaubien, Arthur-Lucien",15


In [13]:
#change the name format into first last with no comma to match MPs_parliament['Full'] column
MPs_parliament['Last'],MPs_parliament['First'] = MPs_parliament['Name'].str.split(',', 1).str
MPs_parliament["First_Last"] = MPs_parliament["First"]  + ' ' + MPs_parliament["Last"] 
MPs_parliament = MPs_parliament.drop(columns=['First', 'Last'])
MPs_parliament.head()

ipykernel_launcher:2: FutureWarning: Columnar iteration over characters will be deprecated in future releases.


,Name,number,First_Last
0,"Beaubien, Arthur-Lucien",14,Arthur-Lucien Beaubien
1,"Beaubien, Arthur-Lucien",15,Arthur-Lucien Beaubien
2,"Beaubien, Arthur-Lucien",16,Arthur-Lucien Beaubien
3,"Beaubien, Arthur-Lucien",17,Arthur-Lucien Beaubien
4,"Beaubien, Arthur-Lucien",18,Arthur-Lucien Beaubien


In [16]:

#used fuzzy matcher to match the names as the format was different
matched_results = fuzzymatcher.fuzzy_left_join(MPs_parliament, All_MPs,
                left_on=['First_Last'],
                right_on=['Full'])


matched_results.head(2)

,best_match_score,__id_left,__id_right,Name_left,number,First_Last,Name_right,Type.of.Parliamentarian,Riding.Senatorial.Division,Province.Territory,Gender,Political.Affiliation,Full,Start_date,End_date,Birth.Date.Combined
0,0.785568,0_left,16_right,"Beaubien, Arthur-Lucien",14,Arthur-Lucien Beaubien,"Beaubien, Arthur-Lucien",MP (1921/12/06 - 1940/01/28)\nSenator (1940/01...,Provencher (1940/01/29 - 1969/02/01)\nProvench...,Manitoba\nManitoba,M,Liberal Party of Canada (1940/01/29 - 1969/02/...,Arthur Beaubien-Lucien,1940-01-29 00:00:00,1969-02-01 00:00:00,"February 11, 1879"
1,0.785568,1_left,16_right,"Beaubien, Arthur-Lucien",15,Arthur-Lucien Beaubien,"Beaubien, Arthur-Lucien",MP (1921/12/06 - 1940/01/28)\nSenator (1940/01...,Provencher (1940/01/29 - 1969/02/01)\nProvench...,Manitoba\nManitoba,M,Liberal Party of Canada (1940/01/29 - 1969/02/...,Arthur Beaubien-Lucien,1940-01-29 00:00:00,1969-02-01 00:00:00,"February 11, 1879"


In [17]:
matched_results.columns

Index(['best_match_score', '__id_left', '__id_right', 'Name_left', 'number',
       'First_Last', 'Name_right', 'Type.of.Parliamentarian',
       'Riding.Senatorial.Division', 'Province.Territory', 'Gender',
       'Political.Affiliation', 'Full', 'Start_date', 'End_date',
       'Birth.Date.Combined'],
      dtype='object')

In [18]:
cols = [
   'best_match_score', 'Name_left',  'Name_right',  'number','__id_left', '__id_right', 
       'First_Last', 'Type.of.Parliamentarian',
       'Riding.Senatorial.Division', 'Province.Territory', 'Gender',
       'Political.Affiliation', 'Full', 'Start_date', 'End_date',
       'Birth.Date.Combined'
]

matched_results[cols].sort_values(by=['best_match_score'], ascending=False).head(5)

,best_match_score,Name_left,Name_right,number,__id_left,__id_right,First_Last,Type.of.Parliamentarian,Riding.Senatorial.Division,Province.Territory,Gender,Political.Affiliation,Full,Start_date,End_date,Birth.Date.Combined
43486,0.990019,"Girard-Bujold, Jocelyne","Girard-Bujold, Jocelyne",36,3985_left,993_right,Jocelyne Girard-Bujold,MP (1997/06/02 - 2004/06/27),Jonquière (1997/06/02 - 2004/06/27),Quebec,F,Bloc Québécois (1997/06/02 - 2004/06/27),Girard-Jocelyne Bujold,1997-06-02 00:00:00,2004-06-27 00:00:00,1943-01-01 00:00:00
45950,0.990019,"Girard-Bujold, Jocelyne","Girard-Bujold, Jocelyne",37,4371_left,993_right,Jocelyne Girard-Bujold,MP (1997/06/02 - 2004/06/27),Jonquière (1997/06/02 - 2004/06/27),Quebec,F,Bloc Québécois (1997/06/02 - 2004/06/27),Girard-Jocelyne Bujold,1997-06-02 00:00:00,2004-06-27 00:00:00,1943-01-01 00:00:00
48225,0.990019,"Girard-Bujold, Jocelyne","Girard-Bujold, Jocelyne",38,4758_left,993_right,Jocelyne Girard-Bujold,MP (1997/06/02 - 2004/06/27),Jonquière (1997/06/02 - 2004/06/27),Quebec,F,Bloc Québécois (1997/06/02 - 2004/06/27),Girard-Jocelyne Bujold,1997-06-02 00:00:00,2004-06-27 00:00:00,1943-01-01 00:00:00
53703,0.975842,"Genest-Jourdain, Jonathan","Genest-Jourdain, Jonathan",42,6344_left,1999_right,Jonathan Genest-Jourdain,MP (2011/05/02 - 2015/10/18),Manicouagan (2011/05/02 - 2015/10/18),Quebec,M,New Democratic Party (2011/05/02 - 2015/10/18),Genest-Jonathan Jourdain,2011-05-02 00:00:00,2015-10-18 00:00:00,1979-07-16 00:00:00
52714,0.975842,"Genest-Jourdain, Jonathan","Genest-Jourdain, Jonathan",41,5914_left,1999_right,Jonathan Genest-Jourdain,MP (2011/05/02 - 2015/10/18),Manicouagan (2011/05/02 - 2015/10/18),Quebec,M,New Democratic Party (2011/05/02 - 2015/10/18),Genest-Jonathan Jourdain,2011-05-02 00:00:00,2015-10-18 00:00:00,1979-07-16 00:00:00


In [27]:
#even low match score seems to be correct, except for John William Bosely, need to check others too
matched_results[cols].query("best_match_score <= .80").sort_values(
    by=['best_match_score'], ascending=True).head(5)

,best_match_score,Name_left,Name_right,number,__id_left,__id_right,First_Last,Type.of.Parliamentarian,Riding.Senatorial.Division,Province.Territory,Gender,Political.Affiliation,Full,Start_date,End_date,Birth.Date.Combined
24243,0.033863,"Baker, Walter David","Walker, David",32,2128_left,1201_right,Walter David Baker,MP (1988/11/21 - 1997/06/01),Winnipeg North Centre (1988/11/21 - 1997/06/01),Manitoba,M,Liberal Party of Canada (1988/11/21 - 1997/06/01),David Walker,1988-11-21 00:00:00,1997-06-01 00:00:00,1947-08-01 00:00:00
15966,0.033863,"Baker, Walter David","Walker, David",30,1402_left,1201_right,Walter David Baker,MP (1988/11/21 - 1997/06/01),Winnipeg North Centre (1988/11/21 - 1997/06/01),Manitoba,M,Liberal Party of Canada (1988/11/21 - 1997/06/01),David Walker,1988-11-21 00:00:00,1997-06-01 00:00:00,1947-08-01 00:00:00
12020,0.033863,"Baker, Walter David","Walker, David",29,1035_left,1201_right,Walter David Baker,MP (1988/11/21 - 1997/06/01),Winnipeg North Centre (1988/11/21 - 1997/06/01),Manitoba,M,Liberal Party of Canada (1988/11/21 - 1997/06/01),David Walker,1988-11-21 00:00:00,1997-06-01 00:00:00,1947-08-01 00:00:00
19799,0.033863,"Baker, Walter David","Walker, David",31,1750_left,1201_right,Walter David Baker,MP (1988/11/21 - 1997/06/01),Winnipeg North Centre (1988/11/21 - 1997/06/01),Manitoba,M,Liberal Party of Canada (1988/11/21 - 1997/06/01),David Walker,1988-11-21 00:00:00,1997-06-01 00:00:00,1947-08-01 00:00:00
33643,0.034859,"Bosley, John William","Williams, John G.",34,2941_left,568_right,John William Bosley,MP (2004/06/28 - 2008/10/13)\nMP (1993/10/25 -...,Edmonton--St. Albert (2004/06/28 - 2008/10/13)...,Alberta\nAlberta,M,Conservative Party of Canada (2003/12/23 - 200...,John Williams,2003-12-23 00:00:00,2008-10-13 00:00:00,1932-02-08 00:00:00
